In [1]:
import os
dataset_path = "/kaggle/input/shrimp-disease-data"

# List files to confirm it's loaded
print(os.listdir(dataset_path))


['Healthy_train', 'white_spots_train', 'Black_gills_train', 'Black_spots_train']


In [2]:
!pip install ultralytics
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.1/922.1 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.6 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Enable mixed precision (optional, speeds up training)
tf.keras.mixed_precision.set_global_policy("mixed_float16")

#  Set Dataset Path (Kaggle)
dataset_path = "/kaggle/input/shrimp-disease-data"

# Image Data Generator (Data Augmentation)
datagen = ImageDataGenerator(
    rescale=1.0/255.0,  
    validation_split=0.2  # 80-20 train-test split
)

#  Load Data (Kaggle format)
img_size = (224, 224)  
batch_size = 32

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

# Load Pretrained ResNet50 (Feature Extraction)
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze ResNet layers

# Add Custom Classification Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x) 
x = Dense(256, activation="relu")(x)
output_layer = Dense(4, activation="softmax")(x)  # 4 disease classes

# Define Model
model = Model(inputs=base_model.input, outputs=output_layer)

#  Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_data.classes),
                                     y=train_data.classes)
class_weights = dict(enumerate(class_weights))


# Train Model
epochs = 100
model.fit(train_data, validation_data=val_data, epochs=epochs, class_weight=class_weights)

# Save Model to Kaggle Output Directory
model.save("/kaggle/working/shrimp_disease_resnet.h5")
print("Model training completed & saved!")

Found 4990 images belonging to 4 classes.
Found 1246 images belonging to 4 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


156/156 ━━━━━━━━━━━━━━━━━━━━ 70s 335ms/step - accuracy: 0.2841 - loss: 1.4235 - val_accuracy: 0.2392 - val_loss: 1.4043
Epoch 2/100
156/156 ━━━━━━━━━━━━━━━━━━━━ 22s 136ms/step - accuracy: 0.3491 - loss: 1.3298 - val_accuracy: 0.2520 - val_loss: 1.3893
Epoch 3/100
156/156 ━━━━━━━━━━━━━━━━━━━━ 22s 134ms/step - accuracy: 0.3738 - loss: 1.3112 - val_accuracy: 0.2552 - val_loss: 1.3932
Epoch 4/100
156/156 ━━━━━━━━━━━━━━━━━━━━ 22s 137ms/step - accuracy: 0.3799 - loss: 1.2855 - val_accuracy: 0.2568 - val_loss: 1.4170
Epoch 5/100
156/156 ━━━━━━━━━━━━━━━━━━━━ 22s 133ms/step - accuracy: 0.4071 - loss: 1.2587 - val_accuracy: 0.2616 - val_loss: 1.3703
Epoch 6/100
156/156 ━━━━━━━━━━━━━━━━━━━━ 22s 135ms/step - accuracy: 0.4225 - loss: 1.2506 - val_accuracy: 0.2833 - val_loss: 1.3690
Epoch 7/100
156/156 ━━━━━━━━━━━━━━━━━━━━ 22s 134ms/step - accuracy: 0.4468 - loss: 1.2200 - val_accuracy: 0.2881 - val_loss: 1.3555
Epoch 8/100
156/156 ━━━━━━━━━━━━━━━━━━━━ 22s 134ms/step - accuracy: 0.4436 - loss: 1.220

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Enable mixed precision (optional, speeds up training)
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# Set Dataset Path (Kaggle)
dataset_path ="/kaggle/input/datashrimp"

# Image Data Generator (Data Augmentation)
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split=0.2  # 80-20 train-test split
)

# Load Data (Kaggle format)
img_size = (224, 224)
batch_size = 32

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

# Load Pretrained ResNet50 (Feature Extraction)
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze all layers initially

# Add Custom Classification Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dense(256, activation="relu")(x)
output_layer = Dense(4, activation="softmax")(x)  # 4 disease classes

# Define Model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile Model (Initial Training)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1.0)  # Gradient Clipping
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_data.classes),
                                     y=train_data.classes)
class_weights = dict(enumerate(class_weights))

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1)
early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Train Model (Initial Feature Extraction Phase)
epochs_initial = 50
model.fit(train_data, validation_data=val_data, epochs=epochs_initial, class_weight=class_weights, callbacks=[reduce_lr, early_stop])

# Fine-Tuning Phase: Unfreeze Last Few Layers
for layer in base_model.layers[-20:]:  # Unfreeze last 20 layers
    layer.trainable = True

# Recompile with a Lower Learning Rate for Fine-Tuning
fine_tune_lr = 1e-5
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=fine_tune_lr, clipnorm=1.0),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Train Model (Fine-Tuning Phase)
epochs_fine_tune = 50
model.fit(train_data, validation_data=val_data, epochs=epochs_fine_tune, class_weight=class_weights, callbacks=[reduce_lr, early_stop])

# Save Final Model
model.save("/kaggle/working/shrimp_disease_resnet_finetuned.h5")
print("Fine-tuning completed & model saved!")


Found 4821 images belonging to 4 classes.
Found 1204 images belonging to 4 classes.
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


151/151 ━━━━━━━━━━━━━━━━━━━━ 63s 334ms/step - accuracy: 0.2862 - loss: 1.3799 - val_accuracy: 0.2085 - val_loss: 1.4490 - learning_rate: 1.0000e-04
Epoch 2/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 20s 128ms/step - accuracy: 0.3514 - loss: 1.3295 - val_accuracy: 0.2757 - val_loss: 1.4064 - learning_rate: 1.0000e-04
Epoch 3/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 21s 133ms/step - accuracy: 0.3930 - loss: 1.2890 - val_accuracy: 0.2724 - val_loss: 1.4077 - learning_rate: 1.0000e-04
Epoch 4/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 21s 132ms/step - accuracy: 0.4007 - loss: 1.2669 - val_accuracy: 0.2450 - val_loss: 1.4106 - learning_rate: 1.0000e-04
Epoch 5/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 21s 133ms/step - accuracy: 0.4055 - loss: 1.2601 - val_accuracy: 0.3023 - val_loss: 1.3740 - learning_rate: 1.0000e-04
Epoch 6/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 21s 133ms/step - accuracy: 0.4443 - loss: 1.2299 - val_accuracy: 0.2533 - val_loss: 1.4122 - learning_rate: 1.0000e-04
Epoch 7/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 21s 131ms/step - 

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Enable mixed precision (optional, speeds up training)
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# Set Dataset Path (Kaggle)
dataset_path = "/kaggle/input/datashrimp"

# Image Data Generator (Data Augmentation)
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2  # 80-20 train-test split
)

# Load Data
img_size = (224, 224)
batch_size = 32

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

# Load Pretrained ResNet50
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Unfreeze last 50 layers for fine-tuning
for layer in base_model.layers[:-50]:  
    layer.trainable = False
for layer in base_model.layers[-50:]:  
    layer.trainable = True

# Add Custom Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)  # Added Dropout
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)  # Added Dropout
output_layer = Dense(4, activation="softmax")(x)  # 4 disease classes

# Define Model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile Model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)  # Smaller LR
model.compile(optimizer=optimizer,
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Callbacks
callbacks = [
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1, min_lr=1e-6),
    EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True, verbose=1)
]

# Train Model
epochs = 50  # Fine-tuning for fewer epochs
history = model.fit(train_data, validation_data=val_data, epochs=epochs, callbacks=callbacks)

# Save Model
model.save("/kaggle/working/shrimp_disease_resnet_finetuned.h5")
print("Fine-tuning completed & model saved!")


Found 4821 images belonging to 4 classes.
Found 1204 images belonging to 4 classes.
Epoch 1/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 123s 606ms/step - accuracy: 0.3060 - loss: 1.3954 - val_accuracy: 0.2600 - val_loss: 1.3913 - learning_rate: 1.0000e-05
Epoch 2/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 75s 479ms/step - accuracy: 0.3471 - loss: 1.3340 - val_accuracy: 0.2932 - val_loss: 1.3971 - learning_rate: 1.0000e-05
Epoch 3/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 74s 473ms/step - accuracy: 0.3843 - loss: 1.3020 - val_accuracy: 0.3156 - val_loss: 1.3871 - learning_rate: 1.0000e-05
Epoch 4/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 75s 476ms/step - accuracy: 0.4228 - loss: 1.2643 - val_accuracy: 0.3405 - val_loss: 1.3692 - learning_rate: 1.0000e-05
Epoch 5/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 76s 486ms/step - accuracy: 0.4342 - loss: 1.2307 - val_accuracy: 0.3239 - val_loss: 1.4206 - learning_rate: 1.0000e-05
Epoch 6/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 75s 475ms/step - accuracy: 0.4490 - loss: 1.2171 - val_accuracy: 0.3671 - val_lo

In [6]:
import cv2
import numpy as np
import albumentations as A
import random
import os
from tqdm import tqdm

# Define dataset paths
INPUT_FOLDER = "/kaggle/input/datashrimp"  # Change to your dataset path
OUTPUT_FOLDER = "/kaggle/working/preprocessed-images"

# Create output directory
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# MixCut: Mix two images randomly
def mixcut(image1, image2, alpha=0.5):
    h, w, _ = image1.shape
    lam = np.random.beta(alpha, alpha)  # Lambda for blending
    x = random.randint(0, w - 1)
    y = random.randint(0, h - 1)
    cut_w = w // 3
    cut_h = h // 3
    image1[y:y+cut_h, x:x+cut_w] = image2[y:y+cut_h, x:x+cut_w]  # Mix patch
    return image1

# Adaptive Histogram Equalization (AHE) for contrast enhancement
def apply_ahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

# Albumentations Pipeline (Color Jitter, Noise)
augmentations = A.Compose([
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    A.GaussNoise(var_limit=(10, 50), p=0.5)
])

# Function to Apply All Preprocessing Steps
def preprocess_image(image1, image2):
    image1 = mixcut(image1, image2)
    image1 = apply_ahe(image1)
    image1 = augmentations(image=image1)['image']
    return image1

# Loop through each class folder
for class_folder in os.listdir(INPUT_FOLDER):
    class_path = os.path.join(INPUT_FOLDER, class_folder)

    if not os.path.isdir(class_path):
        continue  # Skip non-folder files

    # Create corresponding output folder
    output_class_folder = os.path.join(OUTPUT_FOLDER, class_folder)
    os.makedirs(output_class_folder, exist_ok=True)

    # Get all image files
    image_files = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png'))]
    num_images = len(image_files)

    if num_images < 2:
        print(f"Skipping {class_folder}: Not enough images for MixCut.")
        continue

    print(f"Processing {num_images} images in {class_folder}...")

    for i in tqdm(range(0, num_images, 2)):  # Process in pairs
        img1_path = os.path.join(class_path, image_files[i])
        img2_path = os.path.join(class_path, image_files[i+1 if i+1 < num_images else 0])  # Loop last

        image1 = cv2.imread(img1_path)
        image2 = cv2.imread(img2_path)

        if image1 is None or image2 is None:
            print(f"Error reading {img1_path} or {img2_path}, skipping...")
            continue  # Skip if file read fails

        processed_image = preprocess_image(image1, image2)
        output_path = os.path.join(output_class_folder, f"processed_{i}.jpg")
        cv2.imwrite(output_path, processed_image)

print(f"✅ Preprocessing complete! Processed images saved in {OUTPUT_FOLDER}")


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Processing 1501 images in Healthy_train...


100%|██████████| 751/751 [00:24<00:00, 30.32it/s]


Processing 1504 images in white_spots_train...


100%|██████████| 752/752 [00:24<00:00, 30.43it/s]


Processing 1520 images in Black_gills_train...


100%|██████████| 760/760 [00:25<00:00, 30.27it/s]


Processing 1500 images in Black_spots_train...


100%|██████████| 750/750 [00:24<00:00, 30.19it/s]

✅ Preprocessing complete! Processed images saved in /kaggle/working/preprocessed-images
